In [2]:
pip install llama-index-llms-openai llama-index proton_driver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 979.4/979.4 kB 12.7 MB/s eta 0:00:0000:010:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os

from proton_driver import client

timeplus_host = os.getenv("TIMEPLUS_HOST") or "localhost"
timeplus_user = os.getenv("TIMEPLUS_USER") or "proton"
timeplus_password = os.getenv("TIMEPLUS_PASSWORD") or "timeplus@t+"

class Tools:
    def __init__(self) -> None:
        self.client = client.Client(host=timeplus_host, port=8463, user=timeplus_user,password=timeplus_password)

    def list_table(self, *args):
        result = []
        rows = self.client.execute_iter("SHOW STREAMS")
        for row in rows:
            result.append(row[0])
        return result
    
    def describe_table(self, *args):
        name = args[0]
        result = []
        rows = self.client.execute_iter(f"DESCRIBE {name.strip()}")
        for row in rows:
            col = {}
            col["name"] =  row[0]
            col["type"] =  row[1]
            result.append(col)
        return result

    def run(self, tool_name, *args):
        result = getattr(self, tool_name)(*args)
        return result

    def list(self):
        return ["list_table", "describe_table"]

In [13]:
from typing import Annotated
tool = Tools()

def list_table() -> str:
    return tool.list_table()

def describe_table(name: str) -> str:
    return tool.describe_table(name)

list_table_tool = FunctionTool.from_defaults(fn=list_table)
describe_table_tool = FunctionTool.from_defaults(fn=describe_table)

In [4]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool

In [32]:
from llama_index.core import PromptTemplate

react_system_header_str = """\

You are a asistent help generating SQL based on input questions. 
Please stop when you have the SQL, no need to execute the SQL
To generate SQL, here are rules:
* the grammar follows ClickHouse style
* all datatypes MUST be in lowercase, such uint32
* all keywords MUST be in lowercase, such as nullable
* for real time query, where continously return new result to the user, append a time range, for example
  select count(*) from table_name where _tp_time > now() -1h
* for non real time query, add table() function to the table name, for example select count(*) from table(table_name)
  which will return the number of event received in the past 1 hour

## Tools
You have access to a wide variety of tools. You are responsible for using
the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools
to complete each subtask.

You have access to the following tools:
{tool_desc}

## Output Format
To answer the question, please use the following format.

```
Thought: I need to use a tool to help me answer the question.
Action: tool name (one of {tool_names}) if using a tool.
Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {{"input": "hello world", "num_beams": 5}})
```

Please ALWAYS start with a Thought.

Please use a valid JSON format for the Action Input. Do NOT do this {{'input': 'hello world', 'num_beams': 5}}.

If this format is used, the user will respond in the following format:

```
Observation: tool response
```

You should keep repeating the above format until you have enough information
to answer the question without using any more tools. At that point, you MUST respond
in the one of the following two formats:

```
Thought: I can answer without using any more tools.
Answer: [your answer here]
```

```
Thought: I cannot answer the question with the provided tools.
Answer: Sorry, I cannot answer your query.
```

## Additional Rules
- The answer MUST contain a sequence of bullet points that explain how you arrived at the answer. This can include aspects of the previous conversation history.
- You MUST obey the function signature of each tool. Do NOT pass in no arguments if the function expects arguments.

## Current Conversation
Below is the current conversation consisting of interleaving human and assistant messages.

"""
react_system_prompt = PromptTemplate(react_system_header_str)

In [33]:
llm = OpenAI(model="gpt-4o-mini")
agent = ReActAgent.from_tools([list_table_tool, describe_table_tool], llm=llm, verbose=True)
agent.update_prompts({"agent_worker:system_prompt": react_system_prompt})

In [35]:
response = agent.chat("how many customers are there in past three days")

> Running step 798a84e9-95a9-4f6c-830d-b7414165ac32. Step input: how many customers are there in past three days
Thought: (Implicit) I can answer without any more tools!
Answer: ```
select count(*) from table(kafka_cdc_postgres_customers) where _tp_time > now() - 3d
```
- I used the same table 'kafka_cdc_postgres_customers' as it contains customer data.
- I constructed a non-real-time SQL query to count the customers over the past three days by using the table() function.
- I specified the time range to filter the results to the last three days.
